##import

In [1]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow

In [2]:
def Average(lst):
    return sum(lst) / len(lst)

from math import log10, sqrt
  
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

##import_images

In [10]:
path_="/content/drive/MyDrive/NIC_Cropped/"
images=[]
for i in range(100):
  img=cv2.imread(path_+str(i)+'.jpg',0)
  images.append(img)
images=np.array(images)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [11]:
print(type(images[0]))

<class 'numpy.ndarray'>


##filter_codes

In [12]:
def filtering(img, filter_type=2,filter_size=3):
    kernel = np.ones(shape = (filter_size, filter_size))
    x_shape = len(img[0]) 
    y_shape = len(img)

    output = np.zeros_like(img)

    padded_img =  cv2.copyMakeBorder(img, filter_size-2, filter_size-2, filter_size-2, filter_size-2, cv2.BORDER_WRAP)
    # print(padded_img)
    if filter_type==0:
      for x in range(x_shape):
          for y in range(y_shape):
              # applying median filter
              output[y][x] = median(conv(padded_img[  y: y + filter_size, x: x + filter_size]))
      return output
    elif filter_type==1:
      for x in range(x_shape):
          for y in range(y_shape):
              # applying paper filter
              output[y][x] = paper(conv(padded_img[  y: y + filter_size, x: x + filter_size]))

      return output
    elif filter_type==2:
      for x in range(x_shape):
          for y in range(y_shape):
              # applying fuzzy filter
              output[y][x] = fuzzy(conv(padded_img[  y: y + filter_size, x: x + filter_size]))
      return output
    elif filter_type==3:
      for x in range(x_shape):
          for y in range(y_shape):
              # applying median_changed filter
              output[y][x] = median_changed(conv(padded_img[  y: y + filter_size, x: x + filter_size]))
      return output
    else:
      print("error")
      return img



def median(lst):
    sortedLst = sorted(lst)
    return sortedLst[len(lst)//2]

def paper(lst):
    sortedList = sorted(lst)
    L2=[]

    minn=sortedList[0]
    maxx=sortedList[-1]
    avg=sum(sortedList)/9
    median_=sortedList[4]
    for I in sortedList:
      if I==maxx or I==minn:
        L2.append(0)
      elif I==avg:
        L2.append(1)
      elif I<avg:
        L2.append((I-minn)/(avg-minn))
      else:
        L2.append((maxx-I)/(maxx-avg))

    fuzzy_out=sortedList[L2.index(max(L2))]

    return fuzzy_out

def fuzzy(lst):
    sortedLst = sorted(lst)

    for i in range(2):
      z=sortedLst[len(sortedLst)//2]
      for i in range(2):
        x=sortedLst[0]
        y=sortedLst[-1]
        if (z-x)>(y-z):
          sortedLst.remove(x)
        else:
          sortedLst.remove(y)

    return sortedLst[2]

def median_changed(lst):
    sortedLst = sorted(lst)
    lstLen = len(lst)
    index = (lstLen - 1) // 2
    x=(sortedLst[index]-sortedLst[index-1])
    y=sortedLst[index+1]-sortedLst[index]
    if x<y:
      return sortedLst[index]-x/2
    return sortedLst[index]+y/2

def conv(lst):
    result = []
    for i in range(len(lst)):
        for j in range(len(lst[0])):
            result.append(lst[i][j])
    return result

##images_reshape

In [14]:
images_reshaped=[]
for i in images:
  x=i.shape[0]
  y=i.shape[1]
  if x>y:
    images_reshaped.append(cv2.resize(i,(300,400)))
  else:
    images_reshaped.append(cv2.resize(i,(400,300)))

##testing_filters


In [15]:
psnrs=[]
for img in images_reshaped:
  filtered_image=filtering(img,filter_type=2)
  psnr=PSNR(img,filtered_image)
  psnrs.append(psnr)
print(Average(psnrs))

37.03725566194823
